In [6]:
# 3:00
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import os
import urllib.request
import urllib
import random
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

print("="*80)
print("\n")
print("                             유튜브 크롤링")
print("\n")
print("="*80)
print("\n")

query_txt = "유튜브"
query_url = "https://www.youtube.com/"
search_txt = input('''1.크롤링할 검색어를 입력하세요: ''')
count = int(input('''2.몇 건을 가져오겠습니까?: '''))
f_dir = input('3.파일을 저장할 폴더명만 쓰세요.(엔터 입력시 C:/data2/에 저장됨): ')
if f_dir == '':
    f_dir = '/data2/'


print("\n")
print("요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt+'-'+search_txt)
os.chdir(f_dir+s+'-'+query_txt+'-'+search_txt)

ff_dir=f_dir+s+'-'+query_txt+'-'+search_txt
fc_name=f_dir+s+'-'+query_txt+'-'+search_txt+'\\'+s+'-'+query_txt+'-'+search_txt+'.csv'

img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

s_time = time.time( )

chrome_path = "C:/data/chromedriver"
driver = webdriver.Chrome(chrome_path)

driver.get(query_url)
time.sleep(3)

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(search_txt)
element.submit()
time.sleep(2)


def scroll_down(driver):
    driver.execute_script("window.scrollBy(0,3000);")
    time.sleep(5)

if count < 22 :
    page_cnt = 1 
else :
    page_cnt = math.ceil(count/22)

if count > 22 :
    i = 1
    
    while ( i <=  page_cnt) :
        print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(1)
        i += 1

no_list = []
title_list = []
thumbnail_list = []
views_list = []
channel_list = []
img_src2 = []
length_list = []
uldate_list = []
summary_list = []


idx = 0
file_no = 0
no = 1

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

item_list = []
tt = soup.find('ytd-section-list-renderer', 'style-scope ytd-two-column-search-results-renderer')
for t in tt:
    item = t.find('ytd-item-section-renderer', 'style-scope ytd-section-list-renderer')
    item_list.append(item)
    
for i in item:
    
    item2 = i.find_all('ytd-video-renderer')
    

    for j in item2:

        # 번호
        print("-"*80)
        print('번호      : ', no)
        no_list.append(no)
        no += 1

        # 제목
        try:
            title = j.find("yt-formatted-string", "style-scope ytd-video-renderer")
        except:
            title = '제목이 없습니다.'
            print(title.replace("\n",""))
        else:
            title2 = title.get_text()
            print("제목      : ", title2)
            title_list.append(title2)

        # 썸네일 

    #     try :
    #          photo = j.find('img', 'style-scope yt-img-shadow')['src']
    #     except AttributeError :
    #          continue

    #     file_no += 1
    #     urllib.request.urlretrieve(photo,str(file_no)+'.jpg')
    #     time.sleep(0.5)


        # 조회수
        try:
            views = j.find("span", "style-scope ytd-video-meta-block")   
        except:
            views = '조회수가 없습니다.'
            print(views.replace("\n",""))
        else:
            views2 = views.get_text().replace("조회수 ", "")
            print("조회수    : ", views2)
            views_list.append(views2)

        # 길이
        try:
            length = j.find("span", "style-scope ytd-thumbnail-overlay-time-status-renderer")   
        except:
            length = '채널명이 없습니다.'
            print(length.replace("\n",""))
        else:
            length2 = length.get_text().strip()
            print("길이      : ", length2)
            length_list.append(length2)

        # 업로드날짜
        uldate2 = j.select('#metadata-line > span:nth-child(2)')
        for u in range(len(uldate2)):
            uldate3 = uldate2[u].get_text()
            uldate_list.append(uldate3)
            print("업로드날짜: ", uldate3)        

        # 채널명
        try:
            channel = j.find("yt-formatted-string", "style-scope ytd-channel-name")   
        except:
            channel = '채널명이 없습니다.'
            print(channel.replace("\n",""))
        else:
            channel2 = channel.get_text()
            print("채널명    : ", channel2)
            channel_list.append(channel2) 

        # 요약  
        try:
            summary = j.find("yt-formatted-string", "metadata-snippet-text-navigation style-scope ytd-video-renderer")   
        except:
            summary = '요약이 없습니다.'
            print(summary.replace("\n",""))
        else:
            summary2 = summary.get_text()
            print("요약      : ", summary2)
            summary_list.append(summary2)


        if no > count:
            break

        print("-"*80)    
        print("\n")

    #csv로 저장

    df = pd.DataFrame()
    df['번호']=pd.Series(no_list)
    df['제목']=pd.Series(title_list, dtype="string")
    df['조회수']=pd.Series(views_list, dtype="string")
    df['길이']=pd.Series(length_list, dtype="string")
    df['업로드날짜']=pd.Series(uldate_list, dtype="string")
    df['채널명']=pd.Series(channel_list, dtype="string")
    df['요약']=pd.Series(summary_list, dtype="string")



    # 엑셀 형태로 저장하기
    df.to_csv(fc_name, encoding="utf-8-sig",index=False)




    #마지막 요약정보 출력   
    e_time = time.time( )
    t_time = e_time - s_time

    print("\n")
    print("=" *100)
    print("총 소요시간: %s 초" %round(t_time,1))
    print("총 저장 건수: %s 건 " %count)
    print("파일 저장 경로: %s" %fc_name)
    print("=" *100)





                             유튜브 크롤링




1.크롤링할 검색어를 입력하세요: 무한도전
2.몇 건을 가져오겠습니까?: 10
3.파일을 저장할 폴더명만 쓰세요.(엔터 입력시 C:/data2/에 저장됨): 


요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
#이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                 #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fc_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = count+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,count+2) :
    col_name='C'+str(a)
    col_name2.append(col_name)
    time.sleep(0.3)

for b in range(1,count+1) :
    file_name=img_dir+'/'+str(b)+'.jpg'
    file_name2.append(file_name)
    time.sleep(0.3)

for i in range(0,count) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()
    time.sleep(0.3)        
        

In [8]:
# 5:00
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import os
import urllib.request
import urllib
import random
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

print("="*80)
print("\n")
print("                             유튜브 크롤링")
print("\n")
print("="*80)
print("\n")

query_txt = "유튜브"
query_url = "https://www.youtube.com/"
search_txt = input('''1.크롤링할 검색어를 입력하세요: ''')
count = int(input('''2.몇 건을 가져오겠습니까?: '''))
f_dir = input('3.파일을 저장할 폴더명만 쓰세요.(엔터 입력시 C:/data2/에 저장됨): ')
if f_dir == '':
    f_dir = '/data2/'


print("\n")
print("요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요")
print("\n")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt+'-'+search_txt)
os.chdir(f_dir+s+'-'+query_txt+'-'+search_txt)

ff_dir=f_dir+s+'-'+query_txt+'-'+search_txt
fc_name=f_dir+s+'-'+query_txt+'-'+search_txt+'\\'+s+'-'+query_txt+'-'+search_txt+'.csv'

img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

s_time = time.time( )

chrome_path = 'C:/data/chromedriver'
driver = webdriver.Chrome(chrome_path)

driver.get(query_url)
time.sleep(3)

driver.delete_all_cookies()
time.sleep(2)

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(search_txt)
element.submit()
time.sleep(2)

cnt = 1 

def scroll_down(driver):
    driver.execute_script("window.scrollBy(0,3000);")
    time.sleep(5)

if count < 24 :
    page_cnt = 1 
else :
    page_cnt = math.ceil(count/24)

if count > 24 :
    i = 1
    
    while ( i <=  page_cnt) :
        print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(1)
        i += 1

no_list = []
title_list = []
thumbnail_list = []
views_list = []
channel_list = []
img_src2 = []
length_list = []
summary_list = []

file_no = 0
no = 1

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
item = soup.find('ytd-item-section-renderer', 'style-scope ytd-section-list-renderer')
item2 = item.find_all('ytd-video-renderer')

for j in item2:

    # 번호
    print("-"*80)
    print('번호      : ', no)
    no_list.append(no)
    no += 1

    # 제목
    try:
        title = j.find("yt-formatted-string", "style-scope ytd-video-renderer")
    except:
        title = '제목이 없습니다.'
        print(title.replace("\n",""))
    else:
        title2 = title.get_text()
        print("제목      : ", title2)
        title_list.append(title2)

    # 썸네일 

    imgsrc = j.find_elements_by_xpath('//*[@id="img"]')
    print(imgsrc)
    for i, ele in enumerate(imgsrc):
        if i % 2 == 0:
            img_url = ele.get_attribute('src')
            print(img_url)
            file_no += 1
            time.sleep(0.5)
            urllib.request.urlretrieve(img_url,str(file_no)+'.jpg')

    
#     try :
#          photo = j.find('img', 'style-scope yt-img-shadow')['src']
#     except AttributeError :
#          continue
            
#     file_no += 1
#     urllib.request.urlretrieve(photo,str(file_no)+'.jpg')
#     time.sleep(0.5)

    
    # 조회수
    try:
        views = j.find("span", "style-scope ytd-video-meta-block")   
    except:
        views = '조회수가 없습니다.'
        print(views.replace("\n",""))
    else:
        views2 = views.get_text().replace("조회수 ", "")
        print("조회수    : ", views2)
        views_list.append(views2)
       
    # 길이
    try:
        length = j.find("span", "style-scope ytd-thumbnail-overlay-time-status-renderer")   
    except:
        length = '채널명이 없습니다.'
        print(length.replace("\n",""))
    else:
        length2 = length.get_text().strip()
        print("길이      : ", length2)
        length_list.append(length2)
    
    # 업로드날짜
    uldate_list = []
    uldate2 = j.select('#metadata-line > span:nth-child(2)')
    for u in range(len(uldate2)):
        uldate3 = uldate2[u].get_text()
        uldate_list.append(uldate3)
        print("업로드날짜: ", uldate3)        

    # 채널명
    try:
        channel = j.find("yt-formatted-string", "style-scope ytd-channel-name")   
    except:
        channel = '채널명이 없습니다.'
        print(channel.replace("\n",""))
    else:
        channel2 = channel.get_text()
        print("채널명    : ", channel2)
        channel_list.append(channel2) 
    
    # 요약  
    try:
        summary = j.find("yt-formatted-string", "metadata-snippet-text-navigation style-scope ytd-video-renderer")   
    except:
        summary = '요약이 없습니다.'
        print(summary.replace("\n",""))
    else:
        summary2 = summary.get_text()
        print("요약      : ", summary2)
        summary_list.append(summary2)

        
    if no > count:
        break
    
    print("-"*80)    
    print("\n")

#csv로 저장

df = pd.DataFrame()
df['번호']=pd.Series(no_list)
df['제목']=pd.Series(title_list, dtype="string")
df['조회수']=pd.Series(views_list, dtype="string")
df['길이']=pd.Series(length_list, dtype="string")
df['업로드날짜']=pd.Series(uldate_list, dtype="string")
df['채널명']=pd.Series(channel_list, dtype="string")
df['요약']=pd.Series(summary_list, dtype="string")



# 엑셀 형태로 저장하기
df.to_csv(fc_name, encoding="utf-8-sig",index=False)

#이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                 #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fc_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = count+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,count+2) :
    col_name='C'+str(a)
    col_name2.append(col_name)
    time.sleep(0.3)

for b in range(1,count+1) :
    file_name=img_dir+'/'+str(b)+'.jpg'
    file_name2.append(file_name)
    time.sleep(0.3)

for i in range(0,count) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()
    time.sleep(0.3)        
        
        

#마지막 요약정보 출력   
e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *100)
print("총 소요시간: %s 초" %round(t_time,1))
print("총 저장 건수: %s 건 " %count)
print("파일 저장 경로: %s" %fc_name)
print("=" *100)

driver.close( )




                             유튜브 크롤링




1.크롤링할 검색어를 입력하세요: 무한도전
2.몇 건을 가져오겠습니까?: 20
3.파일을 저장할 폴더명만 쓰세요.(엔터 입력시 C:/data2/에 저장됨): 


요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요


--------------------------------------------------------------------------------
번호      :  1
제목      :  [무도] 🐊우리가 알던 메이커가 아니야🐊 주인 없는 집에 사람 채워드림👨‍👩‍👧‍👦 '무한도전 응원단 : 브라질' 8편 MBC140621방송


TypeError: 'NoneType' object is not callable

In [24]:
len('https://yt3.ggpht.com/ytc/')

26

In [ ]:
#이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                 #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
    col_name='C'+str(a)
    col_name2.append(col_name)
    time.sleep(0.3)

for b in range(1,cnt+1) :
    file_name=img_dir+'/'+str(b)+'.jpg'
    file_name2.append(file_name)
    time.sleep(0.3)

for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()
    time.sleep(0.3)